In [1]:
import sys
import os
import glob
import re

import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'sklearn'

In [6]:
df = pd.read_csv('../processed_data/train_v3.csv')
df = df.loc[:,['id', 'age', 'area', 'floor', 'max_floor', 'room_num', 'L', 'D',
       'K', 'S', 'north', 'y']]

In [7]:
df.columns

Index(['id', 'age', 'area', 'floor', 'max_floor', 'room_num', 'L', 'D', 'K',
       'S', 'north', 'y'],
      dtype='object')

In [9]:
X, y = df.loc[:,'age':'north'], df['y']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=0)

NameError: name 'train_test_split' is not defined

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
params = {
        'task' : 'train',
        'boosting_type' : 'gbdt',
        'objective' : 'regression',
        'metric' : 'rmse',
}

In [ ]:
model = lgb.train(params, lgb_train, valid_sets=lgb_eval,num_boost_round=5000)